### Создаем витрины по осищенным данным по расмещенным и выкупленным товарам

In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
from PIL import Image

### Комментарии: 

Данные необходимо смотреть: 

- По всему срезу данных:

Без фильтра по колонке "Status" (только при чистке данных удаляем строки, где "Status" == 'Отменен'. Так как заказы, которые не оформили и мы их не считаем за рассматриваемые и не выкупленные товары: 'Отменен'). 

- По выкупленным товарам:

По выкупленным товарам нужно сделать фильтр по колонке "Status". Значение должно быть равно - 'Выдан клиенту', тогда товар может считаться выкупленным (при чистке данных удаляем строки, где "Status" == 'Отменен'). 

- По чекам с идентификатором клиента:

За идентификатор клиента я беру номер телефона Phone_new. Так же, можно взять Email_new (но телефон являются лучшим способом). После построения процентного содержания значений NaN в слобцах Phone_new и Email_new, мы видим, что значений NaN в этих столбцах нет. Это значит, что все чеки по всему очищенному (!) срезу данных являются чеками с идентификатором клиента. 

#### Выручка | Абсолютная маржа | Маржа | Средняя маржа | Средний чек | Кол-во чеков | Кол-во товаров | Среднее кол-во товаров в  чеке | Кол-во уникальных клиентов | Ценность клиента

- #### МесяцДатыЗаказа
- #### Группа2
- #### Регион        -- 599 уникальных значений
- #### ГородМагазина -- 64 уникальных значений
- #### МагазинЗаказа -- 89 уникальных значений
- #### Гео           -- 'Москва', 'МО', 'Регионы'
- #### МетодДоставки 
- #### ФормаОплаты
- #### Телефон_new (or ЭлектроннаяПочта_new)
- #### ДеньНедели
- #### ВремяОкругл

In [49]:
def generate_table(df, column_name = "МесяцДатыЗаказа"): 
    columns_list = [column_name, 'Выручка', 'Абсолютная маржа', 'Маржа', 'Средняя маржа',
                                 'Средний чек', 'Кол-во чеков', 'Кол-во товаров', 
                                 'Среднее кол-во товаров в  чеке',
                                 'Кол-во уникальных клиентов', 'Ценность клиента', 'Количество'] # add Количество
    
    unique_values = df[column_name].unique()
    total = pd.DataFrame(columns=columns_list)
    
    for val in unique_values:
        
        list_of_values = [val] 
        filtered_df = df[df[column_name] == val]
        
        # Подсчет значений
        
        # Выручка
        profit = int(round(filtered_df['Нов_СуммаСтроки'].sum()))
        list_of_values.append(profit)
        
        # Абсолютная маржа
        absmargin = int(round(filtered_df['Нов_Маржа'].sum()))
        list_of_values.append(absmargin)
        
        # Маржа
        margin = absmargin/profit                                                
        list_of_values.append(margin)
        
        # Средняя маржа !!!
        df_numb_check = filtered_df['НомерЗаказаНаСайте'].unique() # !!!
        numb_check = df_numb_check.size                            # !!!
        absmargin = int(round(filtered_df['Нов_Маржа'].sum()))
        list_of_values.append(absmargin/numb_check) 
        
        # Средний чек
        avarage_bill = int(round(
            filtered_df.groupby(['НомерЗаказаНаСайте']).mean()['Нов_СуммаЗаказаНаСайте'].mean()))
        list_of_values.append(avarage_bill)
        
        # Кол-во чеков !!!
        list_of_values.append(numb_check)
        
        # Кол-во товаров
        numb_good = filtered_df['МесяцДатыЗаказа'].size
        list_of_values.append(numb_good) 
        
        # Среднее кол-во товаров в чеке
        avarage_numb_good = int(round(
            filtered_df.groupby(['НомерЗаказаНаСайте']).count()['Нов_СуммаЗаказаНаСайте'].mean()))
        list_of_values.append(avarage_numb_good)
        
        # Кол-во уникальных клиентов
        num_of_customers = filtered_df['Телефон_new'].unique().size
        list_of_values.append(num_of_customers)

        # Ценность клиента
        value_of_customer = int(round(absmargin /num_of_customers))
        list_of_values.append(value_of_customer) 
        
        # !!!
        # Добавление колонки Количество
        # Количество
        quantity = filtered_df['Количество'].sum()
        list_of_values.append(quantity)
        
        tmp_df = pd.DataFrame([list_of_values], columns=columns_list) 
        total = total.append(tmp_df, ignore_index=True)
        
    return total

### Читаем датасет и добавляем дни недели и время, когда был размещен заказ

In [3]:
df_all = pd.read_csv('for_tables_filtered_data_Размещенные.csv', sep=';')

In [10]:
# День недели
dates = pd.to_datetime(df_all['Дата'])
df_all['Дата_дата'] = dates.dt.date
df_all['ДеньНеделиЦифра'] = dates.dt.dayofweek

# df_all['ДеньНедели'] = dates.dt.weekday_name # old

df_all['ДеньНедели'] = dates.dt.day_name()

# Время - час
df_all['ВремяОкругл'] = dates.dt.hour

### Переименуем дни недели на английском на русский

In [12]:
df_all.loc[df_all['ДеньНедели'] == 'Monday', 'ДеньНедели'] = 'Понедельник'
df_all.loc[df_all['ДеньНедели'] == 'Tuesday', 'ДеньНедели'] = 'Вторник'
df_all.loc[df_all['ДеньНедели'] == 'Wednesday', 'ДеньНедели'] = 'Среда'
df_all.loc[df_all['ДеньНедели'] == 'Thursday', 'ДеньНедели'] = 'Четверг'
df_all.loc[df_all['ДеньНедели'] == 'Friday', 'ДеньНедели'] = 'Пятница'
df_all.loc[df_all['ДеньНедели'] == 'Saturday', 'ДеньНедели'] = 'Суббота'
df_all.loc[df_all['ДеньНедели'] == 'Sunday', 'ДеньНедели'] = 'Воскресенье'

In [ ]:
df_all.shape
df_all


In [15]:
df_all.to_csv('for_tables_filtered_data_Размещенные.csv', index=False, sep=';', encoding='utf-8-sig')

### Фильтры для датасета по всему срезу данных. 

In [32]:
df_all = pd.read_csv('for_tables_filtered_data_Размещенные.csv', sep=';')

In [33]:
df_all = df_all.dropna(subset = ['Группа2'])
df_all = df_all.dropna(subset = ['МетодДоставки'])
df_all = df_all.dropna(subset = ['ФормаОплаты'])

In [34]:
df_all.shape

(435669, 45)

### Pivot tables 

In [27]:
from tabulate import tabulate

def columnValues(df_all):
    row = [df_all.dtypes]
    return row

data = []
for column in df_all:
    data.append([column] + columnValues(df_all[column]))

print(tabulate(data, headers=['data type'], tablefmt='orgtbl'))

|                          | data type   |
|--------------------------+-------------|
| Дата                     | object      |
| ДатаДоставки             | object      |
| НомерЗаказаНаСайте       | object      |
| НовыйСтатус              | object      |
| СуммаЗаказаНаСайте       | object      |
| СуммаДокумента           | float64     |
| МетодДоставки            | object      |
| ФормаОплаты              | object      |
| Регион                   | object      |
| Группа2                  | object      |
| Группа3                  | object      |
| Группа4                  | object      |
| Тип                      | object      |
| Номенклатура             | object      |
| ТипТовара                | object      |
| Отменено                 | object      |
| ПричинаОтмены            | float64     |
| Количество               | int64       |
| Цена                     | float64     |
| СуммаСтроки              | object      |
| ЦенаЗакупки              | float64     |
| МесяцДаты

In [29]:
# df_all['МесяцДатыЗаказа'] = pd.to_datetime(df_all['МесяцДатыЗаказа']) # delete (wrong)
df_all['ДатаДоставки'] = pd.to_datetime(df_all['ДатаДоставки'])
df_all['ДатаЗаказаНаСайте'] = pd.to_datetime(df_all['ДатаЗаказаНаСайте'])
df_all['Дата'] = pd.to_datetime(df_all['Дата'])
# df_all['НовыйСтатус'] = df_all['НовыйСтатус'].astype(str) # str == object # nenuzhno

In [52]:
pivot_table_Month_all = generate_table(df_all, column_name="МесяцДатыЗаказа") 
pivot_table_Month_all.sort_values('Количество', ascending = False)

,МесяцДатыЗаказа,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента,Количество
0,5,210350562,68756796,0.326868,1002.811913,3352,68564,233971,3,54041,1272,363351
1,6,181828009,56876389,0.312803,992.053112,3467,57332,201698,4,47322,1202,315664


In [53]:
pivot_table_Month_all = generate_table(df_all, column_name="МесяцДатыЗаказа") 
pivot_table_Month_all.sort_values('Выручка', ascending = False) 

,МесяцДатыЗаказа,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента,Количество
0,5,210350562,68756796,0.326868,1002.811913,3352,68564,233971,3,54041,1272,363351
1,6,181828009,56876389,0.312803,992.053112,3467,57332,201698,4,47322,1202,315664


In [54]:
pivot_table_Group2_all = generate_table(df_all, column_name="Группа2") 
pivot_table_Group2_all.sort_values('Выручка', ascending = False)

,Группа2,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента,Количество
0,КРУПНОГАБАРИТНЫЙ ТОВАР,141007351,41468068,0.294084,1831.224023,7243,22645,26982,1,20601,2013,27611
9,ПОДГУЗНИКИ,61525215,19921339,0.323791,627.700759,3243,31737,44601,1,23662,842,65462
3,ИГРУШКИ,59909523,11804986,0.197047,312.665166,2784,37756,83943,2,32623,362,88971
1,"ТЕКСТИЛЬ, ТРИКОТАЖ",45578589,14888800,0.326662,544.639134,3262,27337,82403,3,20826,715,85751
10,ДЕТСКОЕ ПИТАНИЕ,38511598,24843785,0.645099,1210.533791,3423,20523,92272,4,14230,1746,274762
5,ТОВАРЫ ДЛЯ КОРМЛЕНИЯ,14725744,2780075,0.188790,199.488734,3562,13936,26137,2,12223,227,28830
2,ОБУВЬ,12831795,4602477,0.358678,514.876049,3207,8939,13405,1,7708,597,13554
8,КОСМЕТИКА/ГИГИЕНА,10276670,3018224,0.293697,196.832138,3507,15334,42774,3,12610,239,55568
4,"КАНЦТОВАРЫ, КНИГИ, ДИСКИ",4118704,1307677,0.317497,244.151792,3119,5356,15607,3,4894,267,20017
12,ТОВАРЫ ДЛЯ ЖИВОТНЫХ,3259459,829146,0.254381,417.075453,2955,1988,5409,3,1634,507,16152


In [55]:
pivot_table_CityShop_all = generate_table(df_all, column_name="ГородМагазина") 
pivot_table_CityShop_all.sort_values('Выручка', ascending = False) 

,ГородМагазина,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента,Количество
0,ИнтернетДоставка,362946424,116440469,0.320820,946.208914,3231,123060,432368,4,88203,1320,675651
5,Москва,3442422,1237683,0.359538,3071.173697,9496,403,486,1,389,3182,498
12,Калуга,2310402,720645,0.311913,3515.341463,11301,205,243,1,169,4264,244
2,Ростов-на-Дону,2133833,557282,0.261165,3028.706522,12048,184,206,1,131,4254,206
6,Саратов,1568992,519232,0.330933,3532.190476,11133,147,211,1,128,4056,225
...,...,...,...,...,...,...,...,...,...,...,...,...
38,Волжский,70471,17616,0.249975,1468.000000,5934,12,12,1,11,1601,13
58,Реутов,67046,22889,0.341392,3269.857143,9578,7,10,1,7,3270,10
63,Коломна,48226,17892,0.371003,2982.000000,8038,6,6,1,6,2982,6
61,Казань,41132,11857,0.288267,1976.166667,6855,6,6,1,6,1976,6


In [56]:
pivot_table_Geo_all = generate_table(df_all, column_name="Гео") 
pivot_table_Geo_all.sort_values('Выручка', ascending = False) 

,Гео,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента,Количество
2,Регионы,217780140,66990567,0.307606,981.532388,3432,68251,193609,3,48686,1376,278801
0,Москва,129415266,44354082,0.342727,1024.225424,3337,43305,184341,4,32707,1356,312494
1,МО,44983164,14288536,0.317642,996.411158,3477,14340,57719,4,10313,1385,87720


In [57]:
pivot_table_DeliveryType_all = generate_table(df_all, column_name="МетодДоставки") 
pivot_table_DeliveryType_all.sort_values('Выручка', ascending = False) 

,МетодДоставки,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента,Количество
0,Магазины,284383901,85116342,0.299301,933.682258,3428,91162,291357,3,67496,1261,408309
2,Курьерская,65789923,24400804,0.370890,1574.855041,4655,15494,90490,6,12928,1887,171954
5,Самовывоз,23140247,8668195,0.374594,620.220020,1657,13976,33073,2,10405,833,58811
1,Pick point,8006937,2878792,0.359537,901.030360,2799,3195,10051,3,2829,1018,15627
3,DPD,6361529,2343166,0.368334,1926.945724,5931,1216,5550,5,1145,2046,10158
4,Транспортная компания,4496033,2225886,0.495078,2609.479484,5451,853,5148,6,712,3126,14156


In [58]:
pivot_table_PaymentType_all = generate_table(df_all, column_name="ФормаОплаты") 
pivot_table_PaymentType_all.sort_values('Выручка', ascending = False) 

,ФормаОплаты,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента,Количество
0,Безналичная,303248413,92564187,0.305242,959.940546,3457,96427,312107,3,71862,1288,448251
1,Наличная,88930158,33068998,0.371854,1122.162204,3233,29469,123562,4,23267,1421,230764


In [59]:
pivot_table_WeekDay_all = generate_table(df_all, column_name="ДеньНедели")
pivot_table_WeekDay_all.sort_values('Выручка', ascending = False)

,ДеньНедели,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента,Количество
2,Среда,70629444,21735806,0.307744,931.627706,3313,23331,84102,4,20789,1046,127241
6,Вторник,65089158,20831728,0.320049,959.456890,3296,21712,77648,4,19496,1069,119168
1,Воскресенье,60188302,19821800,0.329330,1049.327687,3464,18890,62882,3,17026,1164,103589
0,Четверг,58189535,18504355,0.318001,981.038861,3392,18862,66586,4,17177,1077,100693
4,Понедельник,51976531,17178265,0.330500,1060.517657,3498,16198,53436,3,14881,1154,87969
3,Пятница,46024018,14663470,0.318605,991.578983,3392,14788,50587,3,13531,1084,78199
5,Суббота,40081583,12897760,0.321788,1064.610813,3591,12115,40428,3,11214,1150,62156


In [60]:
pivot_table_TimeHours_all = generate_table(df_all, column_name="ВремяОкругл")
pivot_table_TimeHours_all.sort_values('Выручка', ascending = False)

,ВремяОкругл,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента,Количество
14,14,31631665,10299833,0.325618,1048.435770,3534,9824,32726,3,9282,1110,51607
2,15,29712202,9419073,0.317010,1006.526288,3471,9358,31743,3,8834,1066,48780
13,16,29476445,9444687,0.320415,1017.746444,3470,9280,32822,4,8776,1076,51160
16,19,28418562,9465323,0.333068,1078.055011,3515,8780,30281,3,8284,1143,50745
12,17,28149141,9076873,0.322456,1040.210062,3525,8726,31818,4,8250,1100,50336
11,12,26806402,8676342,0.323667,1083.729953,3622,8006,27690,3,7601,1141,44079
15,18,26170789,8466366,0.323504,1045.875973,3551,8095,28121,3,7670,1104,43582
6,11,25499686,8180805,0.320820,1021.323970,3468,8010,26536,3,7588,1078,41709
17,20,23173202,7360363,0.317624,998.692402,3405,7370,25537,3,6994,1052,40674
1,10,23040732,7222172,0.313452,977.157624,3388,7391,24609,3,6969,1036,36969


In [61]:
pivot_table_Phone_new_all = generate_table(df_all, column_name="Телефон_new") 
pivot_table_Phone_new_all 

### Сохраняем

In [ ]:
pivot_table_Phone_new_all.to_csv('pivot_table_Phone_new_all.csv', index=False, sep=';', encoding='utf-8-sig')

In [ ]:
pivot_table_Month_all.to_csv('pivot_table_Month_all.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_Group2_all.to_csv('pivot_table_Group2_all.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_CityShop_all.to_csv('pivot_table_CityShop_all.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_Geo_all.to_csv('pivot_table_Geo_all.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_DeliveryType_all.to_csv('pivot_table_DeliveryType_all.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_PaymentType_all.to_csv('pivot_table_PaymentType_all.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_WeekDay_all.to_csv('pivot_table_WeekDay_all.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_TimeHours_all.to_csv('pivot_table_TimeHours_all.csv', index=False, sep=';', encoding='utf-8-sig')

# По выкупленным товарам. 

### Ориентируемся по 'Новому статусу'. Оставляем только строчки с 'Новым Статусом' == 'Доставлен'. 

В 'Статус' = ‘Доставлен’ есть два подварианта: 'НовыйСтатус' = ‘К отгрузке’ и 'НовыйСтатус' = 'Доставлен'. 
Мы выбираем 'Статус' = ‘Доставлен’: 'НовыйСтатус' = 'Доставлен', значит, должны проверить условие только для 'Нового Статуса'. 

Если мы будем основываться на доставленных товарах, то избежим пропусков в таких полях как 'Дата Доставки', 'Форма оплаты', 'Город доставки'.

In [108]:
df_buyed = df_all[df_all['НовыйСтатус'] == 'Доставлен']

In [109]:
df_buyed.shape

(509572, 45)

**Удалить** все строки, где **есть ПричинаОтмены**, так как тогда товар не может считаться доставленным. 

**Удалить** все строки, где **не Группа2 и Группа3**, так как тогда товар не имеет товарного направлния ("категории") и товарной категории ("подкатегории"). 

In [110]:
# Nan заменим на не сущесвующее значение, чтобы потом удалить все строки не с этим значением 

df_buyed['ПричинаОтмены'] = df_buyed['ПричинаОтмены'].fillna(999999); 
df_buyed = df_buyed[(df_buyed['ПричинаОтмены'] == 999999)] 

<ipython-input-110-7151cf13d660>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_buyed['ПричинаОтмены'] = df_buyed['ПричинаОтмены'].fillna(999999);


In [111]:
df_buyed = df_buyed.dropna(subset = ['Группа2']) 
df_buyed = df_buyed.dropna(subset = ['Группа3']) 

In [112]:
df_buyed.shape

(509572, 45)

In [113]:
df_buyed.to_csv('for_tables_filtered_data_Выкупленные.csv', index=False, sep=';', encoding='utf-8-sig')

### Pivot tables 

In [114]:
pivot_table_Month_buyed = generate_table(df_buyed, column_name="МесяцДатыЗаказа") 
pivot_table_Month_buyed.sort_values('Выручка', ascending = False) 

,МесяцДатыЗаказа,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента
3,8,131725057,35848959,0.272150,804.834965,3072,44542,143864,3,38320,936
0,5,131559550,40288981,0.306241,971.169845,3251,41485,128166,3,34759,1159
2,7,123838443,36163654,0.292023,908.041330,3218,39826,123130,3,34242,1056
1,6,116746415,34118913,0.292248,914.936928,3229,37291,114412,3,32137,1062


In [115]:
pivot_table_Group2_buyed = generate_table(df_buyed, column_name="Группа2") 
pivot_table_Group2_buyed.sort_values('Выручка', ascending = False)

,Группа2,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента
0,КРУПНОГАБАРИТНЫЙ ТОВАР,199015306,54734675,0.275027,1935.660608,7679,28277,33158,1,25572,2140
9,ПОДГУЗНИКИ,79322595,25567431,0.322322,625.196992,2898,40895,58017,1,26683,958
5,ИГРУШКИ,74069279,13459232,0.181711,275.527278,2324,48849,104735,2,41199,327
1,"ТЕКСТИЛЬ, ТРИКОТАЖ",57368930,16007196,0.279022,471.174050,2765,33973,100867,3,25857,619
10,ДЕТСКОЕ ПИТАНИЕ,34340643,22204278,0.646589,1159.976909,3175,19142,76502,4,12650,1755
4,ТОВАРЫ ДЛЯ КОРМЛЕНИЯ,18231143,3238369,0.177628,186.316610,3079,17381,32289,2,14722,220
2,ОБУВЬ,15807185,3996646,0.252837,351.260854,2272,11378,17039,1,9757,410
6,КОСМЕТИКА/ГИГИЕНА,12454732,3279467,0.263311,174.143320,2872,18832,50978,3,14711,223
3,"КАНЦТОВАРЫ, КНИГИ, ДИСКИ",7746400,2457974,0.317305,310.899823,2452,7906,24774,3,7132,345
12,ТОВАРЫ ДЛЯ ЖИВОТНЫХ,4875441,1256662,0.257754,429.334472,2584,2927,8030,3,2078,605


In [116]:
pivot_table_CityShop_buyed = generate_table(df_buyed, column_name="ГородМагазина") 
pivot_table_CityShop_buyed.sort_values('Выручка', ascending = False) 

,ГородМагазина,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента
0,ИнтернетДоставка,455872130,132128636,0.289837,833.761183,2975,158473,504204,3,108608,1217
11,Калуга,4472445,1273598,0.284765,3121.563725,11054,408,469,1,323,3943
2,Ростов-на-Дону,3119593,796734,0.255397,2876.296029,11262,277,314,1,200,3984
5,Москва,2877330,975263,0.338947,2513.564433,7751,388,437,1,370,2636
6,Саратов,2459261,733044,0.298075,3080.016807,10333,238,311,1,210,3491
...,...,...,...,...,...,...,...,...,...,...,...
25,Нижневартовск,152458,51547,0.338106,2577.350000,7623,20,25,1,19,2713
34,Балаково,142590,46681,0.327379,3112.066667,9506,15,19,1,14,3334
60,Казань,118208,33090,0.279930,1946.470588,6953,17,17,1,16,2068
42,Стерлитамак,97292,30993,0.318557,2817.545455,8845,11,11,1,10,3099


In [117]:
pivot_table_Geo_buyed = generate_table(df_buyed, column_name="Гео") 
pivot_table_Geo_buyed.sort_values('Выручка', ascending = False) 

,Гео,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента
2,Регионы,331382983,94618532,0.285526,1008.167455,3654,93852,276075,3,67050,1411
0,Москва,115016457,34659662,0.301345,714.087438,2439,48537,160947,3,33753,1027
1,МО,57470025,17142312,0.298283,825.936497,2837,20755,72550,3,13670,1254


In [118]:
pivot_table_DeliveryType_buyed = generate_table(df_buyed, column_name="МетодДоставки") 
pivot_table_DeliveryType_buyed.sort_values('Выручка', ascending = False) 

,МетодДоставки,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента
0,Магазины,503869465,146420507,0.290592,897.492442,3189,163144,509572,3,112595,1300


In [123]:
pivot_table_PaymentType_buyed = generate_table(df_buyed, column_name="ФормаОплаты") 
pivot_table_PaymentType_buyed.sort_values('Выручка', ascending = False) 

,ФормаОплаты,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента
0,Безналичная,503869465,146420507,0.290592,897.492442,3189,163144,509572,3,112595,1300


In [124]:
pivot_table_WeekDay_buyed = generate_table(df_buyed, column_name="ДеньНедели")
pivot_table_WeekDay_buyed.sort_values('Выручка', ascending = False)

,ДеньНедели,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента
6,Вторник,88366341,25766170,0.291584,851.971365,3026,30243,98053,3,26732,964
2,Среда,84658727,23689618,0.279825,826.862757,3055,28650,93522,3,25650,924
0,Четверг,76193593,21930512,0.287826,888.054748,3184,24695,77751,3,22444,977
1,Воскресенье,71921545,21123082,0.293696,972.517587,3410,21720,65301,3,19855,1064
4,Понедельник,63422422,19119384,0.301461,941.841576,3221,20300,59503,3,18695,1023
3,Пятница,61174002,17528119,0.286529,884.276006,3187,19822,61135,3,18245,961
5,Суббота,58132835,17263622,0.296969,974.575025,3382,17714,54307,3,16388,1053


In [125]:
pivot_table_TimeHours_buyed = generate_table(df_buyed, column_name="ВремяОкругл")
pivot_table_TimeHours_buyed.sort_values('Выручка', ascending = False)

,ВремяОкругл,Выручка,Абсолютная маржа,Маржа,Средняя маржа,Средний чек,Кол-во чеков,Кол-во товаров,Среднее кол-во товаров в чеке,Кол-во уникальных клиентов,Ценность клиента
12,14,38946214,11377747,0.292140,946.883073,3345,12016,36522,3,11364,1001
13,15,35514645,10394343,0.292678,934.406958,3288,11124,33236,3,10567,984
11,16,34925284,10197127,0.291970,913.311867,3225,11165,34838,3,10584,963
10,17,34166984,9871666,0.288924,907.405644,3249,10879,34033,3,10300,958
15,19,33982178,10031799,0.295208,964.317889,3377,10403,31475,3,9892,1014
8,12,31631170,9282298,0.293454,954.085518,3347,9729,28844,3,9237,1005
14,18,31536143,9232680,0.292765,930.995261,3276,9917,31138,3,9441,978
2,13,30390159,8841073,0.290919,935.860379,3323,9447,28422,3,9009,981
9,11,30219160,8844501,0.292679,930.510363,3275,9505,27345,3,9003,982
16,20,29224201,8400589,0.287453,906.603605,3254,9266,28953,3,8882,946


In [ ]:
pivot_table_Phone_new_buyed = generate_table(df_buyed, column_name="Телефон_new") 
pivot_table_Phone_new_buyed

### Сохраняем

In [ ]:
pivot_table_Phone_new_buyed.to_csv('pivot_table_Phone_new_buyed.csv', index=False, sep=';', encoding='utf-8-sig')

In [126]:
pivot_table_Month_buyed.to_csv('pivot_table_Month_buyed.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_Group2_buyed.to_csv('pivot_table_Group2_buyed.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_CityShop_buyed.to_csv('pivot_table_CityShop_buyed.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_Geo_buyed.to_csv('pivot_table_Geo_buyed.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_DeliveryType_buyed.to_csv('pivot_table_DeliveryType_buyed.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_PaymentType_buyed.to_csv('pivot_table_PaymentType_buyed.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_WeekDay_buyed.to_csv('pivot_table_WeekDay_buyed.csv', index=False, sep=';', encoding='utf-8-sig')
pivot_table_TimeHours_buyed.to_csv('pivot_table_TimeHours_buyed.csv', index=False, sep=';', encoding='utf-8-sig')